In [41]:
import pandas as pd
import numpy as np
import pickle
from tensorflow.contrib.learn import preprocessing
from keras.utils.data_utils import get_file

## Pull the raw data and look at a few examples

In [2]:
fname = 'europarl-test.zip'
floc = 'https://storage.googleapis.com/google-code-archive-downloads/v2/code.google.com/language-detection/'
fpath = get_file(fname,floc+fname)
print fpath

/Users/ostegmaier/.keras/datasets/europarl-test.zip


In [4]:
!unzip /Users/ostegmaier/.keras/datasets/europarl-test.zip

Archive:  /Users/ostegmaier/.keras/datasets/europarl-test.zip
  inflating: europarl.test           


In [5]:
with open('europarl.test', 'r') as f:
    rawtxt = f.readlines()

#Show a few raw examples
for t in rawtxt[:3]:
    print t

bg	"Европа 2020" не трябва да стартира нов конкурентен маратон и изход с приватизация.

bg	(CS) Най-голямата несправедливост на сегашната обща селскостопанска политика е фактът, че субсидиите се разпределят неравностойно и несправедливо между старите и новите държави-членки.

bg	(DE) Г-жо председател, г-н член на Комисията, по принцип съм против въвеждането на нови данъци.



### Convert to Numpy Arrays and Reshape to Our desired format

In [6]:
data = np.array([t.split('\t') for t in rawtxt])
data = np.hsplit(data, 2) 
labels = data[0].reshape(1,-1)[0]
text = data[1].reshape(1,-1)[0]

In [7]:
print labels.shape
print text.shape

(21000,)
(21000,)


## Map Test Set to the Corresponding Words From our Vocab

In [14]:
max_doc_len = 71
vocab_path = './misc/20160113.vocab'
vocab_processor = preprocessing.VocabularyProcessor(max_doc_len).restore(vocab_path)

In [32]:
X_test = np.array([t for t in vocab_processor.transform(text)])

In [37]:
print X_test.shape

(21000, 71)


## Map Labels to the Ordering Used in Training

In [48]:
cols = ['bg', 'cs', 'da', 'de', 'el', 'en', 'es', 'et', 'fi', 'fr', 'hu', 'it',
         'lt', 'lv', 'nl', 'pl', 'pt', 'ro', 'sk', 'sl', 'sv']

one_hot_labels = pd.get_dummies(labels)
y_test = one_hot_labels[cols].values

In [49]:
print y_test[0]

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [50]:
dataset = {'x':X_test, 'y':y_test}
with open('./misc/startup_test_set.pkl', 'wb') as f:
    pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)